<center>
<img src="../../img/ods_stickers.jpg" />
    
## [mlcourse.ai](https://mlcourse.ai) – Отворен курс за машинно обучение


Автор: [Дмитрий Сергеев](https://github.com/DmitrySerg), Data Scientist @ Zeptolab, преподавател в Центъра по математически финанси в MSU. Преведено от: @borowis. Този материал е предмет на правилата и условията на лиценза [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Безплатното използване е разрешено за всякакви нетърговски цели.

# <center>9. Анализ на времеви редове в Python</center>

Здрасти!

Продължаваме нашия отворен курс за машинно обучение с нова статия за времеви редове.

Нека да разгледаме как да работим с времеви редове в Python: какви методи и модели можем да използваме за прогнозиране, какво е двойно и тройно експоненциално изглаждане, какво да правите, ако стационарността не е любимото ви нещо, как да изградите SARIMA и да останете живи , как да правите прогнози с помощта на xgboost... В допълнение, всичко това ще бъде приложено към (сурови) примери от реалния свят.

# Описание на статията:
1. [Въведение](#Въведение)
   - [Показатели за качество на прогнозата](#Forecast-quality-metrics)
2. [Преместване, изглаждане, оценка] (#Преместване,-изглаждане,-оценяване)
   - Оценки на подвижния прозорец
   - Експоненциално изглаждане, модел на Holt-Winters
   - Кръстосано валидиране на времеви серии, избор на параметри
3. [Иконометричен подход](#Econometric-approach)
   - Стационарност, единичен корен
   - Отърваване от нестационарността
   - Интуиция и моделиране на SARIMA
4. [Линейни (и не съвсем) модели за времеви серии](#Linear-(and-not-quite)-models-for-time-series)
   - [Извличане на функции] (#Извличане на функции)
   - [Закъснения във времеви серии](#Time-series-lags)
   - [Целево кодиране](#Target-encoding)
   - [Регулиране и избор на функции](#Регулиране-и-избор на функции)
   - [Подсилване] (#Подсилване)
5. [Заключение](#Заключение)
6. [Демо задание](#Демо-задаване)
7. [Полезни ресурси](#Полезни-ресурси)

В ежедневната си работа почти всеки ден се сблъсквам със задачи, свързани с времеви серии. Най-често задаваните въпроси са следните: какво ще се случи с нашите показатели през следващия ден/седмица/месец/и т.н., колко потребители ще инсталират нашето приложение, колко време ще прекарат онлайн, колко действия ще извършат потребителите, и така нататък. Можем да подходим към тези задачи за прогнозиране, като използваме различни методи в зависимост от необходимото качество на прогнозата, продължителността на прогнозния период и, разбира се, времето, в рамките на което трябва да изберем функции и да настроим параметрите, за да постигнем желаните резултати.


# Въведение

Започваме с проста [дефиниция](https://en.wikipedia.org/wiki/Time_series) на времеви редове:
> *Времевата поредица* е поредица от точки от данни, индексирани (или изброени или изобразени на графики) във времеви ред.

Следователно данните са организирани чрез относително детерминистични времеви клейма и могат, в сравнение с произволни примерни данни, да съдържат допълнителна информация, която можем да извлечем.

Нека импортираме някои библиотеки. Първо, ще ни трябва библиотеката [statsmodels](http://statsmodels.sourceforge.net/stable/), която има много функции за статистическо моделиране, включително времеви редове. За любителите на R, които трябваше да преминат към Python, `statsmodels` определено ще изглежда по-познат, тъй като поддържа дефиниции на модели като "Заплата ~ Възраст + Образование".

In [ ]:
import matplotlib.pyplot as plt  # plots
import numpy as np  # vectors and matrices
import pandas as pd  # tables and data manipulations
import seaborn as sns  # more plots

sns.set()

import warnings  
from itertools import product  # some useful functions

import scipy.stats as scs
import statsmodels.api as sm
import statsmodels.formula.api as smf  # statistics and econometrics
import statsmodels.tsa.api as smt
from dateutil.relativedelta import \
    relativedelta  # working with dates with style
from scipy.optimize import minimize  # for function minimization
from tqdm.notebook import tqdm

warnings.filterwarnings("ignore") # `do not disturbe` mode

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Като пример, нека да разгледаме реални данни за мобилни игри. По-конкретно, ще разгледаме гледаните реклами на час и разходите за валута в играта на ден:

In [ ]:
ads = pd.read_csv("../../data/ads.csv", index_col=["Time"], parse_dates=["Time"])
currency = pd.read_csv(
    "../../data/currency.csv", index_col=["Time"], parse_dates=["Time"]
)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(ads.Ads)
plt.title("Ads watched (hourly data)")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(currency.GEMS_GEMS_SPENT)
plt.title("In-game currency spent (daily data)")
plt.grid(True)
plt.show()

## Показатели за качество на прогнозата

Преди да започнем да прогнозираме, нека разберем как да измерваме качеството на нашите прогнози и да разгледаме най-често използваните показатели.

- [R на квадрат](http://scikit-learn.org/stable/modules/model_evaluation.html#r2-score-the-coefficient-of-determination): коефициент на детерминация (в иконометрията това може да се тълкува като процент на дисперсията, обяснен от модела), $(-\infty, 1]$

$R^2 = 1 - \frac{SS_{res}}{SS_{tot}}$

``` python
sklearn.metrics.r2_score
```
---
- [Средна абсолютна грешка](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-error): това е интерпретируема метрика, тъй като има същата мерна единица като първоначалната серия, $[0, +\infty)$

$MAE = \frac{\sum\limits_{i=1}^{n} |y_i - \hat{y}_i|}{n}$

``` python
sklearn.metrics.mean_absolute_error
```
---
- [Средна абсолютна грешка](http://scikit-learn.org/stable/modules/model_evaluation.html#median-absolute-error): отново интерпретируем показател, който е особено интересен, защото е стабилен към извънредни стойности, $[ 0, +\infty)$
$MedAE = median(|y_1 - \hat{y}_1|, ... , |y_n - \hat{y}_n|)$

```python
sklearn.metrics.median_absolute_error
```
---
- [Средна квадратна грешка](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error): най-често използваният показател, който дава по-високо наказание за големи грешки и обратно, $ [0, +\infty)$

$MSE = \frac{1}{n}\sum\limits_{i=1}^{n} (y_i - \hat{y}_i)^2$

``` python
sklearn.metrics.mean_squared_error
```
---
- [Средна квадратна логаритмична грешка](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-logarithmic-error): на практика това е същото като MSE, но вземаме логаритъма от сериала. В резултат на това придаваме по-голяма тежест и на малките грешки. Това обикновено се използва, когато данните имат експоненциални тенденции, $[0, +\infty)$

$MSLE = \frac{1}{n}\sum\limits_{i=1}^{n} (log(1+y_i) - log(1+\hat{y}_i))^2$

``` python
sklearn.metrics.mean_squared_log_error
```
---
- Средна абсолютна процентна грешка: това е същото като MAE, но се изчислява като процент, което е много удобно, когато искате да обясните качеството на модела на ръководството, $[0, +\infty)$

$MAPE = \frac{100}{n}\sum\limits_{i=1}^{n} \frac{|y_i - \hat{y}_i|}{y_i}$

```python
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
```

In [ ]:
# Importing everything from above

from sklearn.metrics import (mean_absolute_error, mean_squared_error,
                             mean_squared_log_error, median_absolute_error,
                             r2_score)


def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

След като вече знаем как да измерваме качеството на прогнозите, нека да видим какви показатели можем да използваме и как да преведем резултатите за шефа. След това остава един малък детайл - изграждане на модела.

# Move, smoothe, evaluate (Преместете, изгладете, оценете)

Да започнем с една наивна хипотеза: "утре ще бъде същото като днес". Въпреки това, вместо модел като $\hat{y}_{t} = y_{t-1}$ (който всъщност е страхотна базова линия за всякакви проблеми с прогнозирането на времеви редове и понякога е невъзможно да се победи), ще приемем, че бъдещата стойност на нашата променлива зависи от средната стойност на нейните $k$ предишни стойности. Затова ще използваме **пълзяща средна**.

$\hat{y}_{t} = \frac{1}{k} \displaystyle\sum^{k}_{n=1} y_{t-n}$

In [ ]:
def moving_average(series, n):
    """
        Calculate average of last n observations
    """
    return np.average(series[-n:])


moving_average(ads, 24)  # prediction for the last observed day (past 24 hours)

За съжаление не можем да правим прогнози далеч в бъдещето - за да получим стойността за следващата стъпка, трябва действително да се наблюдават предишните стойности. Но пълзящата средна има и друг случай на използване - изглаждане на оригиналния времеви ред за идентифициране на тенденции. Pandas има налична реализация с [`DataFrame.rolling(window).mean()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html). Колкото по-широк е прозорецът, толкова по-плавна е тенденцията. В случай на много шумни данни, които често се срещат във финансите, тази процедура може да помогне за откриване на общи модели.

In [ ]:
def plotMovingAverage(
    series, window, plot_intervals=False, scale=1.96, plot_anomalies=False
):

    """
        series - dataframe with timeseries
        window - rolling window size 
        plot_intervals - show confidence intervals
        plot_anomalies - show anomalies 

    """
    rolling_mean = series.rolling(window=window).mean()

    plt.figure(figsize=(15, 5))
    plt.title("Moving average\n window size = {}".format(window))
    plt.plot(rolling_mean, "g", label="Rolling mean trend")

    # Plot confidence intervals for smoothed values
    if plot_intervals:
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bond = rolling_mean - (mae + scale * deviation)
        upper_bond = rolling_mean + (mae + scale * deviation)
        plt.plot(upper_bond, "r--", label="Upper Bond / Lower Bond")
        plt.plot(lower_bond, "r--")

        # Having the intervals, find abnormal values
        if plot_anomalies:
            anomalies = pd.DataFrame(index=series.index, columns=series.columns)
            anomalies[series < lower_bond] = series[series < lower_bond]
            anomalies[series > upper_bond] = series[series > upper_bond]
            plt.plot(anomalies, "ro", markersize=10)

    plt.plot(series[window:], label="Actual values")
    plt.legend(loc="upper left")
    plt.grid(True)

Нека изгладим предходните 4 часа.

In [ ]:
plotMovingAverage(ads, 4)

Сега нека опитаме да изгладим предходните 12 часа.


In [ ]:
plotMovingAverage(ads, 12)

Сега, изглаждайки предходните 24 часа, получаваме дневната тенденция.


In [ ]:
plotMovingAverage(ads, 24)

Когато прилагахме ежедневно изглаждане на почасовите данни, можехме ясно да видим динамиката на гледаните реклами. През уикендите стойностите са по-високи (повече време за игра през уикендите), докато през делничните дни се гледат по-малко реклами.

Можем също така да начертаем доверителни интервали за нашите изгладени стойности.

In [ ]:
plotMovingAverage(ads, 4, plot_intervals=True)

Сега нека създадем проста система за откриване на аномалии с помощта на пълзяща средна. За съжаление, в този конкретен набор от данни всичко е повече или по-малко нормално, така че умишлено ще направим една от стойностите необичайна в нашата рамка с данни „ads_anomaly“.

In [ ]:
ads_anomaly = ads.copy()
ads_anomaly.iloc[-20] = ads_anomaly.iloc[-20] * 0.2  # say we have 80% drop of ads

Нека да видим дали този прост метод може да хване аномалията.

In [ ]:
plotMovingAverage(ads_anomaly, 4, plot_intervals=True, plot_anomalies=True)

Чудесно! Ами втората серия?


In [ ]:
plotMovingAverage(
    currency, 7, plot_intervals=True, plot_anomalies=True
)  # weekly smoothing

О, не, това не беше толкова страхотно! Тук можем да видим недостатъка на нашия прост подход – той не улови месечната сезонност в нашите данни и отбеляза почти всички 30-дневни пикове като аномалии. Ако искате да избегнете фалшиви положителни резултати, най-добре е да помислите за по-сложни модели.

**Претеглена средна стойност** е проста модификация на подвижната средна. Теглата се сумират до „1“ с по-големи тегла, присвоени на по-скорошни наблюдения.


$\hat{y}_{t} = \displaystyle\sum^{k}_{n=1} \omega_n y_{t+1-n}$

In [ ]:
def weighted_average(series, weights):
    """
        Calculate weighted average on the series.
        Assuming weights are sorted in descending order
        (larger weights are assigned to more recent observations).
    """
    result = 0.0
    for n in range(len(weights)):
        result += series.iloc[-n - 1] * weights[n]
    return float(result)

In [ ]:
weighted_average(ads, [0.6, 0.3, 0.1])

In [ ]:
# just checking
0.6 * ads.iloc[-1] + 0.3 * ads.iloc[-2] + 0.1 * ads.iloc[-3]

## Exponential smoothing

Сега, нека да видим какво ще се случи, ако вместо да претегляме последните $k$ стойности на времевия ред, започнем да претегляме всички налични наблюдения, докато експоненциално намаляваме теглата, докато се движим по-назад във времето. Съществува формула за **[експоненциално изглаждане](https://en.wikipedia.org/wiki/Exponential_smoothing)**, която ще ни помогне с това:

$$\hat{y}_{t} = \alpha \cdot y_t + (1-\alpha) \cdot \hat y_{t-1} $$

Тук стойността на модела е среднопретеглена стойност между текущата истинска стойност и стойностите на предишния модел. Теглото $\alpha$ се нарича изглаждащ фактор. Той определя колко бързо ще "забравим" последното налично вярно наблюдение. Колкото по-малък е $\alpha$, толкова по-голямо влияние имат предишните наблюдения и толкова по-плавна е серията.

Експоненциалността се крие в рекурсивността на функцията – ние умножаваме по $(1-\alpha)$ всеки път, което вече съдържа умножение по $(1-\alpha)$ на предишни стойности на модела.

In [ ]:
def exponential_smoothing(series, alpha):
    """
        series - dataset with timestamps
        alpha - float [0.0, 1.0], smoothing parameter
    """
    result = [series[0]]  # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n - 1])
    return result

In [ ]:
def plotExponentialSmoothing(series, alphas):
    """
        Plots exponential smoothing with different alphas
        
        series - dataset with timestamps
        alphas - list of floats, smoothing parameters
        
    """
    with plt.style.context("seaborn-white"):
        plt.figure(figsize=(15, 7))
        for alpha in alphas:
            plt.plot(
                exponential_smoothing(series, alpha), label="Alpha {}".format(alpha)
            )
        plt.plot(series.values, "c", label="Actual")
        plt.legend(loc="best")
        plt.axis("tight")
        plt.title("Exponential Smoothing")
        plt.grid(True);

In [ ]:
plotExponentialSmoothing(ads.Ads, [0.3, 0.05])

In [ ]:
plotExponentialSmoothing(currency.GEMS_GEMS_SPENT, [0.3, 0.05])

## Double exponential smoothing

Досега методите, които обсъждахме, бяха за прогнозиране на една бъдеща точка (с известно изглаждане). Това е готино, но също така не е достатъчно. Нека разширим експоненциалното изглаждане, така че да можем да предвидим две бъдещи точки (разбира се, ще включим и повече изглаждане).

Разлагането на серията ще ни помогне -- получаваме два компонента: отсечка (т.е. ниво) $\ell$ и наклон (т.е. тенденция) $b$. Научихме се да прогнозираме прихващане (или очаквана стойност на серията) с нашите предишни методи; сега ще приложим същото експоненциално изглаждане към тенденцията, като приемем, че бъдещата посока на промените във времевия ред зависи от предишните претеглени промени. В резултат на това получаваме следния набор от функции:

$$\ell_x = \alpha y_x + (1-\alpha)(\ell_{x-1} + b_{x-1})$$

$$b_x = \beta(\ell_x - \ell_{x-1}) + (1-\beta)b_{x-1}$$

$$\hat{y}_{x+1} = \ell_x + b_x$$

Първият описва пресечката, която, както и преди, зависи от текущата стойност на серията. Вторият член вече е разделен на предишни стойности на нивото и на тренда. Втората функция описва тенденцията, която зависи от промените на нивото на текущата стъпка и от предишната стойност на тенденцията. В този случай коефициентът $\beta$ е тегло за експоненциално изглаждане. Крайната прогноза е сумата от стойностите на модела на пресечната точка и тренда.

In [ ]:
def double_exponential_smoothing(series, alpha, beta):
    """
        series - dataset with timeseries
        alpha - float [0.0, 1.0], smoothing parameter for level
        beta - float [0.0, 1.0], smoothing parameter for trend
    """
    # first value is same as series
    result = [series[0]]
    for n in range(1, len(series) + 1):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series):  # forecasting
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha * value + (1 - alpha) * (level + trend)
        trend = beta * (level - last_level) + (1 - beta) * trend
        result.append(level + trend)
    return result


def plotDoubleExponentialSmoothing(series, alphas, betas):
    """
        Plots double exponential smoothing with different alphas and betas
        
        series - dataset with timestamps
        alphas - list of floats, smoothing parameters for level
        betas - list of floats, smoothing parameters for trend
    """

    with plt.style.context("seaborn-white"):
        plt.figure(figsize=(20, 8))
        for alpha in alphas:
            for beta in betas:
                plt.plot(
                    double_exponential_smoothing(series, alpha, beta),
                    label="Alpha {}, beta {}".format(alpha, beta),
                )
        plt.plot(series.values, label="Actual")
        plt.legend(loc="best")
        plt.axis("tight")
        plt.title("Double Exponential Smoothing")
        plt.grid(True)

In [ ]:
plotDoubleExponentialSmoothing(ads.Ads, alphas=[0.9, 0.02], betas=[0.9, 0.02])

In [ ]:
plotDoubleExponentialSmoothing(
    currency.GEMS_GEMS_SPENT, alphas=[0.9, 0.02], betas=[0.9, 0.02]
)

Сега трябва да настроим два параметъра: $\alpha$ и $\beta$. Първият отговаря за изглаждането на серията около тренда, а вторият за изглаждането на самия тренд. Колкото по-големи са стойностите, толкова по-голяма тежест ще имат най-новите наблюдения и толкова по-малко изгладена ще бъде серията от модели. Някои комбинации от параметрите могат да доведат до странни резултати, особено ако са зададени ръчно. След малко ще разгледаме автоматичното избиране на параметри; преди това нека обсъдим тройното експоненциално изглаждане.


## Тройно експоненциално изглаждане, известно още като Holt-Winters

Разгледахме експоненциалното изглаждане и двойното експоненциално изглаждане. Този път преминаваме към _тройно_ експоненциално изглаждане.

Както се досещате, идеята е да се добави и трети компонент – сезонността. Това означава, че не трябва да използваме този метод, ако не се очаква нашите времеви редове да имат сезонност. Сезонните компоненти в модела ще обяснят повтарящите се вариации около отсечката и тенденцията и ще бъдат определени от продължителността на сезона, с други думи от периода, след който вариациите се повтарят. За всяко наблюдение в сезона има отделен компонент; например, ако продължителността на сезона е 7 дни (седмична сезонност), ще имаме 7 сезонни компонента, по един за всеки ден от седмицата.

С това нека напишем нова система от уравнения:

$$\ell_x = \alpha(y_x - s_{x-L}) + (1-\alpha)(\ell_{x-1} + b_{x-1})$$

$$b_x = \beta(\ell_x - \ell_{x-1}) + (1-\beta)b_{x-1}$$

$$s_x = \gamma(y_x - \ell_x) + (1-\gamma)s_{x-L}$$

$$\hat{y}_{x+m} = \ell_x + mb_x + s_{x-L+1+(m-1)modL}$$

Отсечката сега зависи от текущата стойност на серията минус всеки съответен сезонен компонент. Трендът остава непроменен, а сезонният компонент зависи от текущата стойност на серията минус пресечната точка и от предишната стойност на компонента. Имайте предвид, че компонентът се изглажда през всички налични сезони; например, ако имаме компонент понеделник, тогава той ще бъде осреднен само с други понеделници. Можете да прочетете повече за това как работи осредняването и как се извършва първоначалното приближение на тенденцията и сезонните компоненти [тук](http://www.itl.nist.gov/div898/handbook/pmc/section4/pmc435.htm). Сега, след като имаме сезонния компонент, можем да предвидим не само една или две стъпки напред, а произволни $m$ бъдещи стъпки напред, което е много обнадеждаващо.

По-долу е кодът за модел на тройно експоненциално изглаждане, който е известен и с фамилните имена на своите създатели Чарлз Холт и неговия ученик Питър Уинтърс. Освен това, методът Brutlag беше включен в модела за получаване на доверителни интервали:

$$\hat y_{max_x}=\ell_{x−1}+b_{x−1}+s_{x−T}+m⋅d_{t−T}$$

$$\hat y_{min_x}=\ell_{x−1}+b_{x−1}+s_{x−T}-m⋅d_{t−T}$$

$$d_t=\gamma∣y_t−\hat y_t∣+(1−\gamma)d_{t−T},$$

където $T$ е продължителността на сезона, $d$ е предвиденото отклонение. Други параметри са взети от тройно експоненциално изглаждане. Можете да прочетете повече за метода и неговата приложимост за откриване на аномалии във времеви серии [тук](http://fedcsis.org/proceedings/2012/pliks/118.pdf).

In [ ]:
class HoltWinters:

    """
    Модел на Holt-Winters с откриване на аномалии по метода Brutlag
    
    # серия - начална времева серия
    # slen - продължителност на сезона
    # алфа, бета, гама - коефициенти на модела на Холт-Уинтърс
    # n_preds - хоризонт на прогнози
    # scaling_factor - задава ширината на доверителния интервал чрез Brutlag (обикновено приема стойности от 2 до 3)
    
    """

    def __init__(self, series, slen, alpha, beta, gamma, n_preds, scaling_factor=1.96):
        self.series = series
        self.slen = slen
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.n_preds = n_preds
        self.scaling_factor = scaling_factor

    def initial_trend(self):
        sum = 0.0
        for i in range(self.slen):
            sum += float(self.series[i + self.slen] - self.series[i]) / self.slen
        return sum / self.slen

    def initial_seasonal_components(self):
        seasonals = {}
        season_averages = []
        n_seasons = int(len(self.series) / self.slen)
        # let's calculate season averages
        for j in range(n_seasons):
            season_averages.append(
                sum(self.series[self.slen * j : self.slen * j + self.slen])
                / float(self.slen)
            )
        # let's calculate initial values
        for i in range(self.slen):
            sum_of_vals_over_avg = 0.0
            for j in range(n_seasons):
                sum_of_vals_over_avg += (
                    self.series[self.slen * j + i] - season_averages[j]
                )
            seasonals[i] = sum_of_vals_over_avg / n_seasons
        return seasonals

    def triple_exponential_smoothing(self):
        self.result = []
        self.Smooth = []
        self.Season = []
        self.Trend = []
        self.PredictedDeviation = []
        self.UpperBond = []
        self.LowerBond = []

        seasonals = self.initial_seasonal_components()

        for i in range(len(self.series) + self.n_preds):
            if i == 0:  # components initialization
                smooth = self.series[0]
                trend = self.initial_trend()
                self.result.append(self.series[0])
                self.Smooth.append(smooth)
                self.Trend.append(trend)
                self.Season.append(seasonals[i % self.slen])

                self.PredictedDeviation.append(0)

                self.UpperBond.append(
                    self.result[0] + self.scaling_factor * self.PredictedDeviation[0]
                )

                self.LowerBond.append(
                    self.result[0] - self.scaling_factor * self.PredictedDeviation[0]
                )
                continue

            if i >= len(self.series):  # predicting
                m = i - len(self.series) + 1
                self.result.append((smooth + m * trend) + seasonals[i % self.slen])

                # when predicting we increase uncertainty on each step
                self.PredictedDeviation.append(self.PredictedDeviation[-1] * 1.01)

            else:
                val = self.series[i]
                last_smooth, smooth = (
                    smooth,
                    self.alpha * (val - seasonals[i % self.slen])
                    + (1 - self.alpha) * (smooth + trend),
                )
                trend = self.beta * (smooth - last_smooth) + (1 - self.beta) * trend
                seasonals[i % self.slen] = (
                    self.gamma * (val - smooth)
                    + (1 - self.gamma) * seasonals[i % self.slen]
                )
                self.result.append(smooth + trend + seasonals[i % self.slen])

                # Deviation is calculated according to Brutlag algorithm.
                self.PredictedDeviation.append(
                    self.gamma * np.abs(self.series[i] - self.result[i])
                    + (1 - self.gamma) * self.PredictedDeviation[-1]
                )

            self.UpperBond.append(
                self.result[-1] + self.scaling_factor * self.PredictedDeviation[-1]
            )

            self.LowerBond.append(
                self.result[-1] - self.scaling_factor * self.PredictedDeviation[-1]
            )

            self.Smooth.append(smooth)
            self.Trend.append(trend)
            self.Season.append(seasonals[i % self.slen])

## Кръстосано валидиране на времеви редове

Преди да започнем да изграждаме модел, нека първо обсъдим как автоматично да оценяваме параметрите на модела.

Тук няма нищо необичайно; както винаги, трябва да изберем функция на загубите, подходяща за задачата, която ще ни каже колко точно моделът приближава данните. След това, използвайки кръстосано валидиране, ще оценим избраната от нас функция на загуба за дадените параметри на модела, ще изчислим градиента, ще коригираме параметрите на модела и т.н., като в крайна сметка ще се спуснем до глобалния минимум.

Може да питате как да направите кръстосано валидиране за времеви редове, тъй като времевите редове имат тази времева структура и човек не може произволно да смесва стойности в гънка, като запазва тази структура. С рандомизирането всички времеви зависимости между наблюденията ще бъдат загубени. Ето защо ще трябва да използваме по-сложен подход при оптимизиране на параметрите на модела. Не знам дали има официално име за това, но на [CrossValidated](https://stats.stackexchange.com/questions/14099/using-k-fold-cross-validation-for-time-series-model -селекция), където можете да намерите всички отговори, освен отговора на основния въпрос за живота, вселената и всичко, предложеното име за този метод е „кръстосано валидиране на непрекъсната основа“.

Идеята е доста проста -- обучаваме нашия модел на малък сегмент от времевия ред от началото до някои $t$, правим прогнози за следващите $t+n$ стъпки и изчисляваме грешка. След това разширяваме нашата обучителна извадка до $t+n$ стойност, правим прогнози от $t+n$ до $t+2*n$ и продължаваме да преместваме нашия тестов сегмент от времевия ред, докато достигнем последното налично наблюдение. В резултат на това имаме толкова гънки, колкото $n$ ще се поберат между първоначалната тренировъчна извадка и последното наблюдение.

<img src="../../img/time_series_cv.png"/>

Сега, знаейки как да настроим кръстосано валидиране, можем да намерим оптималните параметри за модела на Holt-Winters. Спомнете си, че имаме ежедневна сезонност в рекламите, оттук и параметърът `slen=24`.


In [1]:
from sklearn.model_selection import \
    TimeSeriesSplit  # you have everything done for you


def timeseriesCVscore(params, series, loss_function=mean_squared_error, slen=24):
    """
        Връща грешка в CV
        
        params - вектор от параметри за оптимизация
        серия - набор от данни с времеви серии
        slen - сезонна дължина за модел Holt-Winters
    """
    # errors array
    errors = []

    values = series.values
    alpha, beta, gamma = params

    # set the number of folds for cross-validation
    tscv = TimeSeriesSplit(n_splits=3)

    # iterating over folds, train model on each, forecast and calculate error
    for train, test in tscv.split(values):

        model = HoltWinters(
            series=values[train],
            slen=slen,
            alpha=alpha,
            beta=beta,
            gamma=gamma,
            n_preds=len(test),
        )
        model.triple_exponential_smoothing()

        predictions = model.result[-len(test) :]
        actual = values[test]
        error = loss_function(predictions, actual)
        errors.append(error)

    return np.mean(np.array(errors))

NameError: name 'mean_squared_error' is not defined

В модела на Холт-Уинтърс, както и в другите модели на експоненциално изглаждане, има ограничение за това колко големи могат да бъдат параметрите на изглаждане, като всеки от тях варира от 0 до 1. Следователно, за да минимизираме нашата функция на загубите, ние трябва да изберете алгоритъм, който поддържа ограничения върху параметрите на модела. В нашия случай ще използваме пресечения спрегнат градиент на Нютон.


In [ ]:
%%time
data = ads.Ads[:-20]  # leave some data for testing

# инициализиране на параметрите на модела алфа, бета и гама
x = [0, 0, 0]

# Минимизиране на функцията за загуба
opt = minimize(
    timeseriesCVscore,
    x0=x,
    args=(data, mean_squared_log_error),
    method="TNC",
    bounds=((0, 1), (0, 1), (0, 1)),
)

# Вземете оптимални стойности...
alpha_final, beta_final, gamma_final = opt.x
print(alpha_final, beta_final, gamma_final)

# ...и пуснете train на тях, като прогнозирате за следващите 50 часа
model = HoltWinters(
    data,
    slen=24,
    alpha=alpha_final,
    beta=beta_final,
    gamma=gamma_final,
    n_preds=50,
    scaling_factor=3,
)
model.triple_exponential_smoothing()

Нека добавим малко код за изобразяване на графики.

In [ ]:
def plotHoltWinters(series, plot_intervals=False, plot_anomalies=False):
    """
        series - dataset with timeseries
        plot_intervals - show confidence intervals
        plot_anomalies - show anomalies 
    """

    plt.figure(figsize=(20, 10))
    plt.plot(model.result, label="Model")
    plt.plot(series.values, label="Actual")
    error = mean_absolute_percentage_error(series.values, model.result[: len(series)])
    plt.title("Mean Absolute Percentage Error: {0:.2f}%".format(error))

    if plot_anomalies:
        anomalies = np.array([np.NaN] * len(series))
        anomalies[series.values < model.LowerBond[: len(series)]] = series.values[
            series.values < model.LowerBond[: len(series)]
        ]
        anomalies[series.values > model.UpperBond[: len(series)]] = series.values[
            series.values > model.UpperBond[: len(series)]
        ]
        plt.plot(anomalies, "o", markersize=10, label="Anomalies")

    if plot_intervals:
        plt.plot(model.UpperBond, "r--", alpha=0.5, label="Up/Low confidence")
        plt.plot(model.LowerBond, "r--", alpha=0.5)
        plt.fill_between(
            x=range(0, len(model.result)),
            y1=model.UpperBond,
            y2=model.LowerBond,
            alpha=0.2,
            color="grey",
        )

    plt.vlines(
        len(series),
        ymin=min(model.LowerBond),
        ymax=max(model.UpperBond),
        linestyles="dashed",
    )
    plt.axvspan(len(series) - 20, len(model.result), alpha=0.3, color="lightgrey")
    plt.grid(True)
    plt.axis("tight")
    plt.legend(loc="best", fontsize=13);

In [ ]:
plotHoltWinters(ads.Ads)

In [ ]:
plotHoltWinters(ads.Ads, plot_intervals=True, plot_anomalies=True)

Съдейки по графиките, нашият модел успя успешно да приближи първоначалния времеви ред, като улови дневната сезонност, общата низходяща тенденция и дори някои аномалии. Ако погледнете отклоненията на модела, можете ясно да видите, че моделът реагира доста рязко на промените в структурата на серията, но след това бързо връща отклонението към нормалните стойности, като по същество "забравя" миналото. Тази характеристика на модела ни позволява бързо да изграждаме системи за откриване на аномалии, дори за шумни серии от данни, без да харчим твърде много време и пари за подготовка на данните и обучение на модела.

In [ ]:
plt.figure(figsize=(25, 5))
plt.plot(model.PredictedDeviation)
plt.grid(True)
plt.axis("tight")
plt.title("Brutlag's predicted deviation");

Ще приложим същия алгоритъм за втората серия, която, както може би си спомняте, има тенденция и 30-дневна сезонност.

In [ ]:
%%time
data = currency.GEMS_GEMS_SPENT[:-50]
slen = 30  # 30-day seasonality

x = [0, 0, 0]

opt = minimize(
    timeseriesCVscore,
    x0=x,
    args=(data, mean_absolute_percentage_error, slen),
    method="TNC",
    bounds=((0, 1), (0, 1), (0, 1)),
)

alpha_final, beta_final, gamma_final = opt.x
print(alpha_final, beta_final, gamma_final)

model = HoltWinters(
    data,
    slen=slen,
    alpha=alpha_final,
    beta=beta_final,
    gamma=gamma_final,
    n_preds=100,
    scaling_factor=3,
)
model.triple_exponential_smoothing()

In [ ]:
plotHoltWinters(currency.GEMS_GEMS_SPENT)

Изглежда добре! Моделът улови както възходяща тенденция, така и сезонни пикове и пасва на данните доста добре.

In [ ]:
plotHoltWinters(currency.GEMS_GEMS_SPENT, plot_intervals=True, plot_anomalies=True)

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(model.PredictedDeviation)
plt.grid(True)
plt.axis("tight")
plt.title("Brutlag's predicted deviation");

# Иконометричен подход


### Стационарност

Преди да започнем моделирането, трябва да споменем едно толкова важно свойство на времевите редове: [**стационарност**](https://en.wikipedia.org/wiki/Stationary_process).

Ако даден процес е стационарен, това означава, че той не променя своите статистически свойства с течение на времето, а именно средната стойност и дисперсията. (Постоянството на дисперсията се нарича [хомоскедастичност](https://en.wikipedia.org/wiki/Homoscedasticity)) Ковариационната функция не зависи от времето; трябва да зависи само от разстоянието между наблюденията. Можете да видите това визуално на изображенията в публикацията от [Sean Abu](http://www.seanabu.com/2016/03/22/time-series-seasonal-ARIMA-model-in-python/):

- Червената графика по-долу не е неподвижна, защото средната стойност нараства с времето.

<img src="https://habrastorage.org/files/20c/9d8/a63/20c9d8a633ec436f91dccd4aedcc6940.png"/>

- Нямахме късмет с дисперсията и виждаме различното разпространение на стойностите във времето

<img src="https://habrastorage.org/files/b88/eec/a67/b88eeca676d642449cab135273fd5a95.png"/>

- И накрая, ковариацията на i-тия член и (i + m)-ия член не трябва да бъде функция на времето. В следващата графика ще забележите, че спредът се сближава с времето. Следователно ковариацията не е постоянна с времето в дясната диаграма.

<img src="https://habrastorage.org/files/2f6/1ee/cb2/2f61eecb20714352840748b826e38680.png"/>

И така, защо стационарността е толкова важна? Тъй като е лесно да се правят прогнози за стационарни серии, тъй като можем да приемем, че бъдещите статистически свойства няма да се различават от наблюдаваните в момента. Повечето модели на времеви редове по един или друг начин се опитват да предскажат тези свойства (средна стойност или дисперсия, например). Бъдещите прогнози биха били погрешни, ако оригиналната серия не беше неподвижна. За съжаление повечето времеви редове, които виждаме извън учебниците, са нестационарни, но можем (и трябва) да променим това.

Така че, за да се борим с нестационарността, трябва да познаваме врага си, така да се каже. Да видим как можем да го открием. Ще разгледаме белия шум и произволните разходки, за да научим как да стигнем от един до друг безплатно.

Диаграма на белия шум:

In [ ]:
white_noise = np.random.normal(size=1000)
with plt.style.context("bmh"):
    plt.figure(figsize=(15, 5))
    plt.plot(white_noise)

Процесът, генериран от стандартното нормално разпределение, е стационарен и осцилира около 0 с отклонение от 1. Сега, въз основа на този процес, ще генерираме нов, където всяка следваща стойност ще зависи от предишната: $x_t = \rho x_ {t-1} + e_t$


Ето кода за изобразяване на графиките.

In [ ]:
def plotProcess(n_samples=1000, rho=0):
    x = w = np.random.normal(size=n_samples)
    for t in range(n_samples):
        x[t] = rho * x[t - 1] + w[t]

    with plt.style.context("bmh"):
        plt.figure(figsize=(10, 3))
        plt.plot(x)
        plt.title(
            "Rho {}\n Dickey-Fuller p-value: {}".format(
                rho, round(sm.tsa.stattools.adfuller(x)[1], 3)
            )
        )


for rho in [0, 0.6, 0.9, 1]:
    plotProcess(rho=rho)

На първия график можете да видите същия неподвижен бял шум, както преди. На втория график с $\rho$, увеличен до 0,6, се появиха по-широки цикли, но все още изглежда стационарен като цяло. Третият график се отклонява още повече от средната стойност 0, но все още осцилира около средната стойност. И накрая, с $\rho=1$, имаме процес на произволно ходене, т.е. нестационарен времеви ред.

Това се случва, защото след достигане на критичната стойност серията $x_t = \rho x_{t-1} + e_t$ не се връща към средната си стойност. Ако извадим $x_{t-1}$ от двете страни, ще получим $x_t - x_{t-1} = (\rho - 1) x_{t-1} + e_t$, където изразът отляво се нарича първа разлика. Ако $\rho=1$, тогава първата разлика ни дава стационарен бял шум $e_t$. Това е основната идея зад [теста на Дики-Фулър](https://en.wikipedia.org/wiki/Dickey%E2%80%93Fuller_test) за стационарност на времеви редове (тестване на наличието на единичен корен). Ако можем да получим стационарна серия от нестационарна серия, използвайки първата разлика, ние наричаме тези серии интегрирани от ред 1. Нулевата хипотеза на теста е, че времевият ред е нестационарен, което беше отхвърлено при първите три парцели и накрая приет на последния. Трябва да кажем, че първата разлика не винаги е достатъчна, за да получим стационарна серия, тъй като процесът може да бъде интегриран от порядък d, d > 1 (и да има множество единични корени). В такива случаи се използва разширеният тест на Дики-Фулър, който проверява няколко лага наведнъж.

Можем да се борим с нестационарността, като използваме различни подходи: различни разлики в реда, премахване на тенденция и сезонност, изглаждане и трансформации като Box-Cox или логаритмични.


## Отърваване от нестационарността и изграждане на SARIMA

Нека изградим модел на ARIMA, като преминем през всички ~~кръгове на ада~~ етапи на създаване на серия неподвижна.

Ето кода за изобразяване на графики.

In [ ]:
def tsplot(y, lags=None, figsize=(12, 7), style="bmh"):
    """
        Начертайте времеви редове, техните ACF и PACF, изчислете теста на Дики-Фулър
        
        y - времеви серии
        закъснения(lagg) - колко закъснения да се включат в изчислението на ACF, PACF
    """
    if not isinstance(y, pd.Series):
        y = pd.Series(y)

    with plt.style.context(style):
        fig = plt.figure(figsize=figsize)
        layout = (2, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))

        ts_ax.plot(y)
        p_value = sm.tsa.stattools.adfuller(y)[1]
        ts_ax.set_title(
            "Time Series Analysis Plots\n Dickey-Fuller: p={0:.5f}".format(p_value)
        )
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()

In [ ]:
tsplot(ads.Ads, lags=60)

_този отклонение на диаграмата на частична автокорелация изглежда като грешка в statsmodels, частичната автокорелация трябва да бъде <= 1 като всяка корелация._

Изненадващо, първоначалните серии са неподвижни; тестът на Дики-Фулър отхвърли нулевата хипотеза, че е налице единичен корен. Всъщност можем да видим това на самата графика – нямаме видима тенденция, така че средната стойност е постоянна, а дисперсията е доста стабилна. Единственото, което остава, е сезонността, с която трябва да се справим преди моделирането. За да направим това, нека вземем "сезонната разлика", което означава просто изваждане на серията от самата нея с лаг, който е равен на сезонния период.

In [ ]:
ads_diff = ads.Ads - ads.Ads.shift(24)
tsplot(ads_diff[24:], lags=60)

Сега е много по-добре с изчезналата видима сезонност. Функцията за автокорелация обаче все още има твърде много значителни закъснения. За да ги премахнем, ще вземем първите разлики, като извадим серията от самата нея със закъснение 1.


In [ ]:
ads_diff = ads_diff - ads_diff.shift(1)
tsplot(ads_diff[24 + 1 :], lags=60)

Перфектно! Нашата серия сега изглежда като нещо неописуемо, осцилиращо около нулата. Тестът на Дики-Фулър показва, че е стационарен и броят на значимите пикове в ACF е намалял. Най-накрая можем да започнем да моделираме!

## ARIMA-семеен интензивен курс

Ще обясним този модел, като изграждаме буква по буква. $SARIMA(p, d, q)(P, D, Q, s)$, модел на сезонна авторегресия на подвижната средна:

- $AR(p)$ - авторегресионен модел, т.е. регресия на времевия ред върху себе си. Основното предположение е, че текущите стойности на серията зависят от предишните стойности с известно закъснение (или няколко закъснения). Максималното забавяне в модела се означава като $p$. За да определите първоначалния $p$, трябва да погледнете графиката на PACF и да намерите най-голямото значимо закъснение, след което **повечето** други закъснения стават незначителни.
- $MA(q)$ - модел на пълзяща средна. Без да навлизаме в много подробности, това моделира грешката на времевия ред, отново с предположението, че текущата грешка зависи от предишната с известно закъснение, което се нарича $q$. Първоначалната стойност може да бъде намерена на диаграмата на ACF със същата логика, както преди. 

Нека комбинираме нашите първи 4 букви:

$AR(p) + MA(q) = ARMA(p, q)$

Това, което имаме тук, е моделът на авторегресия – подвижна средна! Ако редицата е неподвижна, тя може да бъде апроксимирана с тези 4 букви. Да продължим.

- $I(d)$ - ред на интегриране. Това е просто броят на несезонните разлики, необходими, за да бъде серията стационарна. В нашия случай това е само 1, защото използвахме първите разлики.

Добавянето на тази буква към четирите ни дава модела $ARIMA$, който може да обработва нестационарни данни с помощта на несезонни разлики. Страхотно, остава още едно писмо!

- $S(s)$ - това отговаря за сезонността и се равнява на продължителността на сезонния период на серията

С това имаме три параметъра: $(P, D, Q)$

- $P$ - ред на авторегресия за сезонния компонент на модела, който може да бъде извлечен от PACF. Но трябва да погледнете броя на значителните закъснения, които са кратни на продължителността на сезонния период. Например, ако периодът е равен на 24 и виждаме, че 24-ият и 48-ият лаг са значими в PACF, това означава, че първоначалният $P$ трябва да бъде 2.

- $Q$ - подобна логика, използваща вместо това графиката на ACF.

- $D$ - ред на сезонно интегриране. Това може да бъде равно на 1 или 0, в зависимост от това дали са приложени сезонни разлики или не.

След като вече знаем как да зададем първоначалните параметри, нека отново да погледнем окончателния график и да зададем параметрите:

In [ ]:
tsplot(ads_diff[24 + 1 :], lags=60)

- $p$ най-вероятно е 4, тъй като това е последното значително забавяне на PACF, след което повечето други не са значими.
- $d$ е равно на 1, защото имахме първи разлики
- $q$ трябва да е някъде около 4, както и да се вижда на ACF
- $P$ може да е 2, тъй като 24-ти и 48-ми лагове са донякъде значими за PACF
- $D$ отново е равно на 1, защото направихме сезонна диференциация
- $Q$ вероятно е 1. 24-то забавяне на ACF е значително, докато 48-ото не е.

Нека тестваме различни модели и да видим кой е по-добър.

In [ ]:
# задаване на начални стойности и някои граници за тях
ps = range(2, 5)
d = 1
qs = range(2, 5)
Ps = range(0, 2)
D = 1
Qs = range(0, 2)
s = 24 # Продължителността на сезона все още е 24
# създаване на списък с всички възможни комбинации от параметри
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [ ]:
def optimizeSARIMA(parameters_list, d, D, s):
    """
        Върнете рамка с данни с параметри и съответния AIC
        
        parameters_list - списък с (p, q, P, Q) кортежи
        d - ред на интегриране в ARIMA модел
        D - ред за сезонно интегриране
        s - продължителност на сезона
    """

    results = []
    best_aic = float("inf")

    for param in tqdm(parameters_list):
    # трябва да опитаме - освен защото при някои комбинации моделът не успява да се сближи
        try:
            model = sm.tsa.statespace.SARIMAX(
                ads.Ads,
                order=(param[0], d, param[1]),
                seasonal_order=(param[2], D, param[3], s),
            ).fit(disp=-1)
        except:
            continue
        aic = model.aic
        # saving best model, AIC and parameters
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = param
        results.append([param, model.aic])

    result_table = pd.DataFrame(results)
    result_table.columns = ["parameters", "aic"]
# сортиране във възходящ ред, колкото по-нисък е AIC - толкова по-добре
    result_table = result_table.sort_values(by="aic", ascending=True).reset_index(
        drop=True
    )

    return result_table

In [ ]:
%%time
result_table = optimizeSARIMA(parameters_list, d, D, s)

In [ ]:
result_table.head()

In [ ]:
# set the parameters that give the lowest AIC
p, q, P, Q = result_table.parameters[0]

best_model = sm.tsa.statespace.SARIMAX(
    ads.Ads, order=(p, d, q), seasonal_order=(P, D, Q, s)
).fit(disp=-1)
print(best_model.summary())

Нека инспектираме остатъците от модела.

In [ ]:
tsplot(best_model.resid[24 + 1 :], lags=60)

Ясно е, че остатъците са стационарни и няма видими автокорелации. Нека направим прогнози, използвайки нашия модел.


In [ ]:
def plotSARIMA(series, model, n_steps):
    """
        Plots model vs predicted values
        
        series - dataset with timeseries
        model - fitted SARIMA model
        n_steps - number of steps to predict in the future
        
    """
    # adding model values
    data = series.copy()
    data.columns = ["actual"]
    data["arima_model"] = model.fittedvalues
    # making a shift on s+d steps, because these values were unobserved by the model
    # due to the differentiating
    data["arima_model"][: s + d] = np.NaN

    # forecasting on n_steps forward
    forecast = model.predict(start=data.shape[0], end=data.shape[0] + n_steps)
    forecast = data.arima_model.append(forecast)
    # calculate error, again having shifted on s+d steps from the beginning
    error = mean_absolute_percentage_error(
        data["actual"][s + d :], data["arima_model"][s + d :]
    )

    plt.figure(figsize=(15, 7))
    plt.title("Mean Absolute Percentage Error: {0:.2f}%".format(error))
    plt.plot(forecast, color="r", label="model")
    plt.axvspan(data.index[-1], forecast.index[-1], alpha=0.5, color="lightgrey")
    plt.plot(data.actual, label="actual")
    plt.legend()
    plt.grid(True);

In [ ]:
plotSARIMA(ads, best_model, 50)

В крайна сметка получихме много адекватни прогнози. Нашият модел греши средно с 4,01%, което е много, много добре. Въпреки това, общите разходи за подготовка на данни, правене на сериите неподвижни и избор на параметри може да не си струват тази точност.


# Линейни (и не съвсем) модели за времеви редове

Често в работата си трябва да създавам модели с [*бързо, добро, евтино*](http://fastgood.cheap) като единствен ръководен принцип. Това означава, че някои от тези модели никога няма да се считат за „готови за производство“, тъй като изискват твърде много време за подготовка на данните (както в SARIMA) или изискват често повторно обучение за нови данни (отново SARIMA) или са трудни за настройка (добре пример - SARIMA). Следователно много често е много по-лесно да изберете няколко функции от съществуващите времеви редове и да изградите прост линеен регресионен модел или, да речем, произволна гора. Добър е и евтин.

Този подход не е подкрепен от теория и нарушава няколко предположения (напр. теоремата на Гаус-Марков, особено за грешките, които не са корелирани), но е много полезен на практика и често се използва в състезания по машинно обучение.

## Извличане на функции

Моделът се нуждае от функции и всичко, което имаме, е едномерен времеви ред. Какви функции можем да извлечем?
* Изоставане на времеви редове
* Статистика на прозореца:
    - Макс./мин. стойност на серията в прозорец
    - Средна/средна стойност в прозорец
    - Дисперсия на прозореца
    - и т.н.
* Функции за дата и час:
    - Минута от час, час от ден, ден от седмицата и т.н
    - Този ден празник ли е? Може би има специално събитие? Представете това като булева характеристика
* Целево кодиране
* Прогнози от други модели (имайте предвид, че по този начин можем да загубим скоростта на прогнозиране)

Нека прегледаме някои от методите и да видим какво можем да извлечем от нашите данни за времевите серии на рекламите.

## Изоставане на времеви редове

Премествайки серията $n$ със стъпки назад, получаваме колона с характеристики, където текущата стойност на времевата серия е подравнена със стойността й в момент $t-n$. Ако направим едно изместване на изоставането и обучим модел на тази характеристика, моделът ще може да прогнозира с 1 стъпка напред, след като е наблюдавал текущото състояние на серията. Увеличаването на забавянето, да речем, до 6, ще позволи на модела да прави прогнози с 6 стъпки напред; обаче ще използва данни, наблюдавани 6 стъпки назад. Ако нещо фундаментално промени серията през този ненаблюдаван период, моделът няма да улови тези промени и ще върне прогнози с голяма грешка. Следователно, по време на първоначалния избор на забавяне, трябва да се намери баланс между оптималното качество на прогнозата и дължината на прогнозния хоризонт.

In [ ]:
# Създаване на копие на първоначалната дейтаграма за извършване на различни трансформации
data = pd.DataFrame(ads.Ads.copy())
data.columns = ["y"]

In [ ]:
# Добавяне на изоставането на целевата променлива от 6 стъпки назад до 24
for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

In [ ]:
# погледнете новата рамка с данни
data.tail(7)

Страхотно, генерирахме набор от данни тук. Защо сега не обучим модел?


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

# за кръстосано валидиране на времеви серии, задайте 5 пъти
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
def timeseries_train_test_split(X, y, test_size):
    
"""
        Извършете разделяне на train-test по отношение на структурата на времевите серии
    """

    # вземете индекса, след който започва тестовият набор
    test_index = int(len(X) * (1 - test_size))

    X_train = X.iloc[:test_index]
    y_train = y.iloc[:test_index]
    X_test = X.iloc[test_index:]
    y_test = y.iloc[test_index:]

    return X_train, X_test, y_train, y_test

In [ ]:
y = data.dropna().y
X = data.dropna().drop(["y"], axis=1)

# запазване на 30% от данните за тестване
X_train, X_test, y_train, y_test = timeseries_train_test_split(X, y, test_size=0.3)

In [ ]:
# машинно обучение в два реда
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
def plotModelResults(
    model, X_train=X_train, X_test=X_test, plot_intervals=False, plot_anomalies=False
):
    
"""
        Графики, моделирани срещу фактически стойности, прогнозни интервали и аномалии
    
    """

    prediction = model.predict(X_test)

    plt.figure(figsize=(15, 7))
    plt.plot(prediction, "g", label="prediction", linewidth=2.0)
    plt.plot(y_test.values, label="actual", linewidth=2.0)

    if plot_intervals:
        cv = cross_val_score(
            model, X_train, y_train, cv=tscv, scoring="neg_mean_absolute_error"
        )
        mae = cv.mean() * (-1)
        deviation = cv.std()

        scale = 1.96
        lower = prediction - (mae + scale * deviation)
        upper = prediction + (mae + scale * deviation)

        plt.plot(lower, "r--", label="upper bond / lower bond", alpha=0.5)
        plt.plot(upper, "r--", alpha=0.5)

        if plot_anomalies:
            anomalies = np.array([np.NaN] * len(y_test))
            anomalies[y_test < lower] = y_test[y_test < lower]
            anomalies[y_test > upper] = y_test[y_test > upper]
            plt.plot(anomalies, "o", markersize=10, label="Anomalies")

    error = mean_absolute_percentage_error(prediction, y_test)
    plt.title("Mean absolute percentage error {0:.2f}%".format(error))
    plt.legend(loc="best")
    plt.tight_layout()
    plt.grid(True)


def plotCoefficients(model):
    """
        Изобразява сортирани стойности на коефициента на модела
    """

    coefs = pd.DataFrame(model.coef_, X_train.columns)
    coefs.columns = ["coef"]
    coefs["abs"] = coefs.coef.apply(np.abs)
    coefs = coefs.sort_values(by="abs", ascending=False).drop(["abs"], axis=1)

    plt.figure(figsize=(15, 7))
    coefs.coef.plot(kind="bar")
    plt.grid(True, axis="y")
    plt.hlines(y=0, xmin=0, xmax=len(coefs), linestyles="dashed");

In [ ]:
plotModelResults(lr, plot_intervals=True)
plotCoefficients(lr)

Обикновените забавяния и линейната регресия ни дадоха прогнози, които не са толкова далеч от SARIMA по отношение на качеството. Има много ненужни функции, така че ще направим избор на функции след малко. Засега нека продължим с инженерството!


Ще добавим час, ден от седмицата и булева стойност за „is_weekend“. За да направим това, трябва да трансформираме текущия индекс на кадъра с данни във формат „datetime“ и да извлечем „hour“ и „weekday“.


In [ ]:
data.index = pd.to_datetime(data.index)
data["hour"] = data.index.hour
data["weekday"] = data.index.weekday
data["is_weekend"] = data.weekday.isin([5, 6]) * 1
data.tail()

Можем да визуализираме получените характеристики.

In [ ]:
plt.figure(figsize=(16, 5))
plt.title("Encoded features")
data.hour.plot()
data.weekday.plot()
data.is_weekend.plot()
plt.grid(True);

Тъй като сега имаме различни скали в нашите променливи, хиляди за характеристиките на изоставането и десетки за категоричните, трябва да ги трансформираме в една и съща скала за изследване на важността на характеристиките и, по-късно, регулиране.


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [ ]:
y = data.dropna().y
X = data.dropna().drop(["y"], axis=1)

X_train, X_test, y_train, y_test = timeseries_train_test_split(X, y, test_size=0.3)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

plotModelResults(lr, X_train=X_train_scaled, X_test=X_test_scaled, plot_intervals=True)
plotCoefficients(lr)

Грешката на теста намалява малко. Съдейки по графиката на коефициентите, можем да кажем, че `weekday` и `is_weekend` са полезни функции.


## Целево кодиране
Бих искал да добавя още един вариант за кодиране на категорични променливи: кодиране по средна стойност. Ако е нежелателно да се разрушава набор от данни чрез използване на много фиктивни променливи, които могат да доведат до загуба на информация и ако те не могат да се използват като реални стойности поради конфликти като „0 часа < 23 часа“, тогава е възможно да се кодира променлива с малко по-интерпретируеми стойности. Естествената идея е да се кодира със средната стойност на целевата променлива. В нашия пример всеки ден от седмицата и всеки час от деня могат да бъдат кодирани от съответния среден брой реклами, гледани през този ден или час. Много е важно да се уверите, че средната стойност се изчислява само върху набора за обучение (или само върху текущото сгъване за кръстосано валидиране), така че моделът да не е наясно с бъдещето.

In [ ]:
def code_mean(data, cat_feature, real_feature):
    """
    Връща речник, където ключовете са уникални категории на cat_feature,
    и стойностите са средни стойности над real_feature
    """
    return dict(data.groupby(cat_feature)[real_feature].mean())

Нека да разгледаме средните стойности по часове.

In [ ]:
average_hour = code_mean(data, "hour", "y")
plt.figure(figsize=(7, 5))
plt.title("Hour averages")
pd.DataFrame.from_dict(average_hour, orient="index")[0].plot()
plt.grid(True);

И накрая, нека съберем всички трансформации в една функция.

In [ ]:
def prepareData(series, lag_start, lag_end, test_size, target_encoding=False):
    """
        серия: pd.DataFrame
            рамка от данни с времеви серии

        lag_start: int
            първоначална стъпка назад във времето за разделяне на целевата променлива
            пример - lag_start = 1 означава, че моделът
                      ще видите вчерашните стойности, за да прогнозирате днес

        lag_end: int
            последна стъпка назад във времето за разделяне на целевата променлива
            пример - lag_end = 4 означава, че моделът
                      ще видите до 4 дни назад във времето, за да прогнозирате днес

        test_size: float
            размер на тестовия набор от данни след разделянето на влак/тест като процент от набора от данни

        target_encoding: boolean
            ако е True - добавете целеви средни стойности към набора от данни
        
    """

    # копие на първоначалния набор от данни
    data = pd.DataFrame(series.copy())
    data.columns = ["y"]

    # закъснения на серия
    for i in range(lag_start, lag_end):
        data["lag_{}".format(i)] = data.y.shift(i)

    # datetime features
    data.index = pd.to_datetime(data.index)
    data["hour"] = data.index.hour
    data["weekday"] = data.index.weekday
    data["is_weekend"] = data.weekday.isin([5, 6]) * 1

    if target_encoding:
    # изчисляване на средни стойности само за train
        test_index = int(len(data.dropna()) * (1 - test_size))
        data["weekday_average"] = list(
            map(code_mean(data[:test_index], "weekday", "y").get, data.weekday)
        )
        data["hour_average"] = list(
            map(code_mean(data[:test_index], "hour", "y").get, data.hour)
        )

    # капка кодирани променливи
        data.drop(["hour", "weekday"], axis=1, inplace=True)

    # train-test split
    y = data.dropna().y
    X = data.dropna().drop(["y"], axis=1)
    X_train, X_test, y_train, y_test = timeseries_train_test_split(
        X, y, test_size=test_size
    )

    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = prepareData(
    ads.Ads, lag_start=6, lag_end=25, test_size=0.3, target_encoding=True
)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

plotModelResults(
    lr,
    X_train=X_train_scaled,
    X_test=X_test_scaled,
    plot_intervals=True,
    plot_anomalies=True,
)
plotCoefficients(lr)

Виждаме някакво **прекомерно оборудване**! „Hour_average“ беше толкова голямо в обучителния набор от данни, че моделът реши да концентрира всичките си сили върху него. В резултат на това качеството на прогнозата падна. Този проблем може да бъде разрешен по различни начини; например можем да изчислим целевото кодиране не за целия набор от влакове, а вместо това за някакъв прозорец. По този начин кодировките от последния наблюдаван прозорец най-вероятно ще опишат по-добре текущото състояние на серията. Като алтернатива можем просто да го пуснем ръчно, тъй като сме сигурни, че това само влошава нещата в този случай.


In [ ]:
X_train, X_test, y_train, y_test = prepareData(
    ads.Ads, lag_start=6, lag_end=25, test_size=0.3, target_encoding=False
)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Регулиране и избор на функции

Както вече знаем, не всички функции са еднакво здрави - някои могат да доведат до прекомерно оборудване, докато други трябва да бъдат премахнати. Освен ръчна проверка, можем да приложим регуляризация. Два от най-популярните регресионни модели с регуляризация са регресиите на Ridge и Lasso. И двете добавят някои допълнителни ограничения към нашата функция за загуба.

В случай на регресия на Ридж, тези ограничения са сумата от квадратите на коефициентите, умножени по коефициента на регулация. Колкото по-голям е коефициентът, толкова по-голяма ще бъде загубата ни. Следователно ще се опитаме да оптимизираме модела, като запазим коефициентите сравнително ниски.

В резултат на тази $L2$ регулация ще имаме по-голямо отклонение и по-ниска вариация, така че моделът ще обобщава по-добре (поне това е, което се надяваме да се случи).

Вторият регресионен модел, Lasso regression, добавя към функцията на загубите не квадрати, а абсолютни стойности на коефициентите. В резултат на това по време на процеса на оптимизация коефициентите на маловажни характеристики могат да станат нули, което позволява автоматизиран избор на характеристики. Този тип регулация се нарича $L1$.

Първо, нека се уверим, че имаме функции за премахване и че данните имат силно корелирани характеристики.


In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(X_train.corr());

In [ ]:
from sklearn.linear_model import LassoCV, RidgeCV

ridge = RidgeCV(cv=tscv)
ridge.fit(X_train_scaled, y_train)

plotModelResults(
    ridge,
    X_train=X_train_scaled,
    X_test=X_test_scaled,
    plot_intervals=True,
    plot_anomalies=True,
)
plotCoefficients(ridge)

Можем ясно да видим, че някои коефициенти се доближават все повече и повече до нула (въпреки че всъщност никога не я достигат), тъй като значението им в модела пада.


In [ ]:
lasso = LassoCV(cv=tscv)
lasso.fit(X_train_scaled, y_train)

plotModelResults(
    lasso,
    X_train=X_train_scaled,
    X_test=X_test_scaled,
    plot_intervals=True,
    plot_anomalies=True,
)
plotCoefficients(lasso)

Ласо регресията се оказа по-консервативна; премахна 23-то изоставане от най-важните функции и напълно премахна 5 характеристики, което само подобри качеството на прогнозиране.

# Подсилване
Защо не трябва да опитаме XGBoost сега?
<img src="../../img/xgboost_the_things.jpg"/>

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(verbosity=0)
xgb.fit(X_train_scaled, y_train);

In [ ]:
plotModelResults(
    xgb,
    X_train=X_train_scaled,
    X_test=X_test_scaled,
    plot_intervals=True,
    plot_anomalies=True,
)

Имаме победител! Това е най-малката грешка в тестовата серия сред всички модели, които сме пробвали досега.

Но тази победа е решаваща и може би не е най-добрата идея да поставите „xgboost“ веднага щом се докопате до данни от времеви серии. Като цяло, дървовидните модели се справят лошо с тенденциите в данните в сравнение с линейните модели. В такъв случай първо ще трябва да премахнете тренда на сериала си или да използвате някои трикове, за да направите магията да се случи. В идеалния случай можете да направите серията неподвижна и след това да използвате XGBoost. Например, можете да прогнозирате тенденция отделно с линеен модел и след това да добавите прогнози от „xgboost“, за да получите окончателна прогноза.

# Заключение

Обсъдихме различни методи за анализ на времеви редове и прогнози. За съжаление или може би за щастие, няма един единствен начин за разрешаване на този тип проблеми. Методите, разработени през 60-те години (а някои дори в началото на 21-ви век), са все още популярни, заедно с LSTM и RNN (не са обхванати в тази статия). Това е отчасти свързано с факта, че задачата за прогнозиране, както всяка друга задача, свързана с данни, изисква креативност в толкова много аспекти и определено изисква проучване. Въпреки големия брой формални показатели за качество и подходи за оценка на параметрите, често е необходимо да се опита нещо различно за всеки времеви ред. Не на последно място важен е балансът между качество и цена. Като добър пример, моделът SARIMA може да доведе до впечатляващи резултати след настройка, но може да изисква много часове манипулиране на времеви серии ~~танци на тамбура~~, докато прост линеен регресионен модел може да бъде изграден за 10 минути и може да постигне повече или по-малко сравними резултати.

# Полезни ресурси

* Същият бележник като интерактивен уеб базиран [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-9-part-1-time-series-analysis-in-python)
* „LSTM (Long Short Term Memory) Networks for predicting Time Series“ – урок от Макс Сергей Булаев в mlcourse.ai (пълният списък с уроци е [тук](https://mlcourse.ai/tutorials))
* Основно ястие [сайт](https://mlcourse.ai), [репо за курс](https://github.com/Yorko/mlcourse.ai) и YouTube [канал](https://www.youtube. com/watch?v=QKTuw4PNOsU&list=PLVlY_7IJCMJeRfZ68eVfEcu-UcN9BbwiX)
* Материали за курса като [Набор от данни на Kaggle](https://www.kaggle.com/kashnitsky/mlcourse)
* Среден ["история"](https://medium.com/open-machine-learning-course/open-machine-learning-course-topic-9-time-series-analysis-in-python-a270cb05e0b3?source= collection_home---6------2--------------------) въз основа на този бележник
* Ако четете руски: [статия](https://habr.com/ru/company/ods/blog/327242/) на Habr.com със ~ същия материал. И [лекция](https://youtu.be/_9lBwXnbOd8) в YouTube
* [Онлайн учебник](https://people.duke.edu/~rnau/411home.htm) за курса за разширено статистическо прогнозиране в университета Дюк - обхваща подробно различни техники за изглаждане заедно с линейни и ARIMA модели
* [Сравнение на моделите на времевите серии ARIMA и Random Forest за прогнозиране на огнища на птичи грип H5N1] (https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-15-276) - един от малкото случаи, при които се използва случайна гора за прогнозиране на времеви редове се защитава активно
* [Анализ на времеви серии (TSA) в Python - линейни модели към GARCH](http://www.blackarbs.com/blog/time-series-analysis-in-python-linear-models-to-garch/11/1 /2016) - прилагане на фамилията модели ARIMA към задачата за моделиране на финансови показатели (от Браян Кристофър)